In [1]:
### I've added my comments with triple # ~ Chris

# This is just something to start with. If there's anything you wish to change, feel free to do so.
# If there's anything that needs clarifying please indicate it as a comment with #### prefixed.

import numpy as np
import time
import matplotlib.pyplot as plt
import random


# note: instances should be read from text file instead of as defined below

allowableTime = 2 # time allowed in seconds
p = 1 + np.random.randint(1000, size = 612) # job processing times
p = np.append(p, 1 + np.random.randint(2001, size = 132))
p = np.append(p, 1 + 1 + np.random.negative_binomial(n = 1500, p = 0.5, size = 201))
m = 201 # number of machines
inst = np.append(allowableTime,np.append(m,p))

### creating an example txt instance

with open("instance.txt", "w") as f:
    print(inst, file = f)
    
### Given an instance will be of the form (𝑝1,𝑝2,⋯,𝑝𝑛,𝑚) we need to read the txt file in this way. We can then chop it up for the agent to manipulate

def import_inst(filename):
    '''
    imports a text file instance, converts it to an array and then allocates it to p and m, 
    where p are the jobs and m is the number of machines
    '''
    inst = list(map(int, re.findall('\d+', str([line.rstrip('\n') for line in open(filename)]))))
    global p, m, allowableTime
    m =  int(inst[2:])
    p = inst[1]
    allowableTime = inst[0]
### - FIXED. This will now work for gibberish inputs of any alphabet


# makespan instance from Charl's lecture notes (part 3)
# allowableTime = 1
# m = 4
# p = [3, 2, 4, 1, 3, 3, 6]

# this defines an 'agent' object which would implement a heuristic to solve the makespan problem
# note: for neatness, this should later be moved to its own file
class agent:
    def __init__(self): 
        self.allocation = {x: [] for x in range(m)} ### A dictionary where we can keep track of which job is assigned to which machine ie: allocation[0] = [k_1, ...] are the jobs assigned to the first machine
        self.machine = [] # list of length len(p), where self.machine[job] = machine assigned to job ### lets discuss whether this variable is necessary
        self.workload = np.array([]) # np.array of length m, where self.workload[machine] = sum of processing times of jobs assigned to machine
        self.cost = None # cost of current feasible solution
        self.costTrajectory = [] # list of cost of feasible solution found in each step
    # generates a random initial feasible solution
    def generateInitialSolution(self):
        self.machine = []
        self.workload = np.zeros(m)
        for job in range(len(p)):
            machine = np.random.randint(m) # randomly select machine to assign job to 
            ### should we have different variable name given that 'machine' is already an attribute ?
            self.machine.append(machine) # assign 'job' to 'machine'
            self.workload[machine] += p[job] # add job processing time to workload of 'machine'
        self.cost = np.max(self.workload)
    # switch assigned machine of 'job' to 'toMachine'
    def switchMachine(self, job, toMachine):
        self.workload[self.machine[job]] += -p[job]
        self.machine[job] = toMachine
        self.workload[toMachine] += p[job]
    # Note that this is just a (random) local search implementation, not greedy local search as question 1 asks
    def localSearchIteration(self, k):
        costAlpha = self.cost
        jobs = [int(len(p)*random.random()) for i in range(k)] # randomly choose k jobs to reassign
        machines = [int(m*random.random()) for i in range(k)] # randomly choose k machines to assign these jobs to
        initialMachines = [self.machine[i] for i in jobs]
        for (job, machine) in zip(jobs, machines):
            self.switchMachine(job, machine)
        costBeta = self.workload[np.argmax(self.workload)] # find max workload across machines, faster than np.max(self.workload) 
        # if new feasible solution is worse then go back, otherwise stay
        if costBeta > costAlpha:
            for (job, machine) in zip(jobs, initialMachines):
                self.switchMachine(job, machine)
            self.costTrajectory.append(costAlpha)
        else:
            self.cost = costBeta
            self.costTrajectory.append(costBeta)
    # k defines the size of the neighbourhood and totalTime determines how much time the function is allowed to run
    def localSearch(self, k, totalTime):
        initialTime = time.time()
        count = 0
        # note: algorithm may run later than cut-off if iteration takes too long!
        while time.time() - initialTime < totalTime:
            self.localSearchIteration(k)
            count += 1
            if count %1000 == 0:
                print(np.var(self.workload))
        ### print out the resulting workload
        print(self.workload)
        ### maybe a bar plot/histogram might give a good representation of how well we are progressing toward optimal
        plt.bar(range(1,m+1),self.workload)
        plt.xlabel("machines")
        plt.ylabel("workload")
        plt.show()
    
    ### we'll begin with a rigid definition of neighbour, where a neighbour is "machine +/- 1 mod len(m)"
    ### we'll construct it in a similar fashion to our random local search
    def generateGreedySolution(self):
        ### couple of different ideas about how to sort this array - 
        ### see "https://stackoverflow.com/questions/26984414/efficiently-sorting-a-numpy-array-in-descending-order"
        ### to begin with, i'll go with an inplace sorting, but whether this is efficent we'll discuss later
        self.machine = []
        self.workload = np.zeros(m)
        p[::-1].sort()
        for i in range(len(p)):
            worker = np.argmin(self.workload)
            self.allocation[worker].append(i)
            self.machine.append(worker)
            self.workload[worker] += p[i]
        self.cost = np.max(self.workload)
        print(self.workload)
### insert new code here
    
A = agent()
# A.generateInitialSolution()
A.generateGreedySolution()
A.localSearch(4, allowableTime)

plt.plot(A.costTrajectory)
plt.xlabel("iteration")
plt.ylabel("cost of feasible solution")
plt.axhline(y = max(sum(p)/m, np.max(p)), color = "gold") # lower bound for global minimum




[3642. 3641. 3612. 3625. 3676. 3673. 3681. 3648. 3660. 3659. 3781. 3764.
 3747. 3758. 3753. 3647. 3746. 3720. 3744. 3713. 3622. 3604. 3608. 3603.
 3621. 3712. 3741. 3730. 3738. 3763. 3693. 3646. 3707. 3690. 3707. 3762.
 3751. 3794. 3737. 3790. 3736. 3787. 3775. 3761. 3783. 3749. 3775. 3726.
 3736. 3726. 3774. 3747. 3760. 3689. 3712. 3719. 3646. 3707. 3676. 3725.
 3723. 3687. 3719. 3658. 3687. 3669. 3679. 3706. 3705. 3686. 3711. 3699.
 3668. 3735. 3759. 3699. 3774. 3768. 3677. 3722. 3734. 3765. 3697. 3721.
 3719. 3709. 3683. 3638. 3707. 3675. 3632. 3632. 3682. 3654. 3676. 3626.
 3622. 3621. 3624. 3646. 3644. 3623. 3627. 3640. 3627. 3621. 3620. 3624.
 3624. 3607. 3606. 3620. 3618. 3617. 3615. 3613. 3622. 3606. 3602. 3603.
 3602. 3601. 3602. 3602. 3603. 3609. 3599. 3603. 3603. 3599. 3603. 3602.
 3602. 3603. 3602. 3602. 3602. 3608. 3628. 3627. 3668. 3654. 3731. 3711.
 3695. 3637. 3681. 3650. 3778. 3782. 3665. 3681. 3716. 3617. 3611. 3626.
 3634. 3680. 3695. 3695. 3697. 3695. 3694. 3697. 37

<Figure size 640x480 with 1 Axes>

In [19]:
# determines whether the solution found by A is indeed feasible, input: A is an 'agent' object
def verifyFeasibleSolution(A):
    # check that each job is assigned to exactly one machine
    assert(len(A.machine) == len(p))
    # check that there are at most m machines that have jobs assigned to them
    assert(max(A.machine) <= m)

    # check that the workloads are as indicated in A.workload
    workload = np.zeros(m)
    for job in range(len(p)):
        workload[A.machine[job]] += p[job]
    for i in range(m):
        assert(workload[i] == A.workload[i])

    # check that the maximum of the workloads (i.e. the cost) is as indicated in A.cost
    assert(np.isclose(A.cost, np.max(A.workload)))

verifyFeasibleSolution(A)

AttributeError: 'numpy.ndarray' object has no attribute 'machine'

In [ ]:
### we'll begin with a rigid definition of neighbour, where a neighbour is "machine +/- 1 mod len(m)"
    ### we'll construct it in a similar fashion to our random local search
def generateGreedySolution(self):
    ### couple of different ideas about how to sort this array - 
    ### see "https://stackoverflow.com/questions/26984414/efficiently-sorting-a-numpy-array-in-descending-order"
    ### to begin with, i'll go with an inplace sorting, but whether this is efficent we'll discuss later
    self.machine = []
    self.workload = np.zeros(m)
    p[::-1].sort()
    for i in range(len(p)):
        worker = np.argmin(self.workload)
        self.machine.append(worker)
        self.workload[worker] += p[i]
    self.cost = np.max(self.workload)
    print(self.workload)
    
def greedylocalSearchIteration(self):
    

In [ ]:
Allocation

In [51]:
class test_agent:
    def __init__(self): 
        self.allocation = {x: [] for x in range(m)} ### A dictionary where we can keep track of which job is assigned to which machine ie: allocation[0] = [k_1, ...] are the jobs assigned to the first machine
        self.workload = np.array([]) # np.array of length m, where self.workload[machine] = sum of processing times of jobs assigned to machine
        self.cost = None # cost of current feasible solution
        self.costTrajectory = [] # list of cost of feasible solution found in each step
    # generates a random initial feasible solution
    def generateGreedySolution(self):
        ### couple of different ideas about how to sort this array - 
        ### see "https://stackoverflow.com/questions/26984414/efficiently-sorting-a-numpy-array-in-descending-order"
        ### to begin with, i'll go with an inplace sorting, but whether this is efficent we'll discuss later
        self.workload = np.zeros(m)
        p[::-1].sort()
        for i in range(len(p)):
            worker = np.argmin(self.workload)
            self.allocation[worker].append(p[i])
            self.workload[worker] += p[i]
        self.cost = np.max(self.workload)
        print(self.workload)
### insert new code here
    
A = test_agent()
A.generateGreedySolution()


[3642. 3641. 3612. 3625. 3676. 3673. 3681. 3648. 3660. 3659. 3781. 3764.
 3747. 3758. 3753. 3647. 3746. 3720. 3744. 3713. 3622. 3604. 3608. 3603.
 3621. 3712. 3741. 3730. 3738. 3763. 3693. 3646. 3707. 3690. 3707. 3762.
 3751. 3794. 3737. 3790. 3736. 3787. 3775. 3761. 3783. 3749. 3775. 3726.
 3736. 3726. 3774. 3747. 3760. 3689. 3712. 3719. 3646. 3707. 3676. 3725.
 3723. 3687. 3719. 3658. 3687. 3669. 3679. 3706. 3705. 3686. 3711. 3699.
 3668. 3735. 3759. 3699. 3774. 3768. 3677. 3722. 3734. 3765. 3697. 3721.
 3719. 3709. 3683. 3638. 3707. 3675. 3632. 3632. 3682. 3654. 3676. 3626.
 3622. 3621. 3624. 3646. 3644. 3623. 3627. 3640. 3627. 3621. 3620. 3624.
 3624. 3607. 3606. 3620. 3618. 3617. 3615. 3613. 3622. 3606. 3602. 3603.
 3602. 3601. 3602. 3602. 3603. 3609. 3599. 3603. 3603. 3599. 3603. 3602.
 3602. 3603. 3602. 3602. 3602. 3608. 3628. 3627. 3668. 3654. 3731. 3711.
 3695. 3637. 3681. 3650. 3778. 3782. 3665. 3681. 3716. 3617. 3611. 3626.
 3634. 3680. 3695. 3695. 3697. 3695. 3694. 3697. 37

In [56]:
A.allocation

{0: [1991, 799, 552, 300],
 1: [1985, 803, 552, 301],
 2: [1948, 807, 554, 303],
 3: [1902, 815, 556, 308, 44],
 4: [1887, 816, 557, 310, 106],
 5: [1883, 817, 560, 310, 103],
 6: [1876, 824, 557, 311, 113],
 7: [1876, 822, 564, 310, 76],
 8: [1871, 825, 565, 310, 89],
 9: [1868, 827, 567, 309, 88],
 10: [1846, 828, 568, 314, 225],
 11: [1834, 829, 572, 324, 205],
 12: [1823, 832, 572, 334, 186],
 13: [1822, 832, 573, 333, 198],
 14: [1818, 833, 573, 336, 193],
 15: [1798, 834, 580, 360, 75],
 16: [1774, 834, 582, 371, 185],
 17: [1765, 834, 582, 382, 157],
 18: [1757, 837, 588, 379, 183],
 19: [1735, 839, 589, 401, 149],
 20: [1678, 839, 597, 478, 30],
 21: [1639, 840, 632, 483, 10],
 22: [1637, 843, 628, 485, 15],
 23: [1632, 843, 646, 473, 9],
 24: [1630, 844, 650, 466, 31],
 25: [1614, 848, 712, 391, 147],
 26: [1614, 847, 716, 384, 180],
 27: [1611, 848, 734, 369, 168],
 28: [1610, 849, 733, 369, 177],
 29: [1608, 852, 730, 369, 204],
 30: [1600, 854, 759, 354, 126],
 31: [1598, 8

In [58]:
test = A.allocation[0]

In [59]:
test.remove(1991)

In [60]:
test

[799, 552, 300]

In [68]:
### I've added my comments with triple # ~ Chris

# This is just something to start with. If there's anything you wish to change, feel free to do so.
# If there's anything that needs clarifying please indicate it as a comment with #### prefixed.

import numpy as np
import time
import matplotlib.pyplot as plt
import random


# note: instances should be read from text file instead of as defined below

allowableTime = 2 # time allowed in seconds
p = 1 + np.random.randint(1000, size = 612) # job processing times
p = np.append(p, 1 + np.random.randint(2001, size = 132))
p = np.append(p, 1 + 1 + np.random.negative_binomial(n = 1500, p = 0.5, size = 201))
m = 201 # number of machines
inst = np.append(allowableTime,np.append(m,p))

### creating an example txt instance

with open("instance.txt", "w") as f:
    print(inst, file = f)
    
### Given an instance will be of the form (𝑝1,𝑝2,⋯,𝑝𝑛,𝑚) we need to read the txt file in this way. We can then chop it up for the agent to manipulate

def import_inst(filename):
    '''
    imports a text file instance, converts it to an array and then allocates it to p and m, 
    where p are the jobs and m is the number of machines
    '''
    inst = list(map(int, re.findall('\d+', str([line.rstrip('\n') for line in open(filename)]))))
    global p, m, allowableTime
    m =  int(inst[2:])
    p = inst[1]
    allowableTime = inst[0]

# this defines an 'agent' object which would implement a heuristic to solve the makespan problem
# note: for neatness, this should later be moved to its own file
class agent:
    def __init__(self): 
        self.allocation = {x: [] for x in range(m)} ### A dictionary where we can keep track of which job is assigned to which machine ie: allocation[0] = [k_1, ...] are the jobs assigned to the first machine
        self.workload = np.array([]) # np.array of length m, where self.workload[machine] = sum of processing times of jobs assigned to machine
        self.cost = None # cost of current feasible solution
        self.costTrajectory = [] # list of cost of feasible solution found in each step
        
    # generates a greedy initial feasible solution
    def generateGreedySolution(self):
        ### couple of different ideas about how to sort this array - 
        ### see "https://stackoverflow.com/questions/26984414/efficiently-sorting-a-numpy-array-in-descending-order"
        ### to begin with, i'll go with an inplace sorting, but whether this is efficent we'll discuss later
        self.workload = np.zeros(m)
        p[::-1].sort()
        for i in range(len(p)):
            worker = np.argmin(self.workload)
            self.allocation[worker].append(p[i])
            self.workload[worker] += p[i]
        self.cost = np.max(self.workload)
        print(self.workload)
        
    # switch assigned machine of 'job' to 'toMachine'
    def switchMachine(self,  job, fromMachine, toMachine):
        self.workload[fromMachine] += - job
        self.allocation[fromMachine].remove(job)
        self.workload[toMachine] += job
        self.allocation[toMachine].append(job)
            
    

    ### the switch machine function will work fine for the greedy search
    
        
A = agent()





In [69]:
A.generateGreedySolution()

[3648. 3658. 3660. 3667. 3673. 3694. 3753. 3781. 3752. 3779. 3776. 3712.
 3750. 3697. 3696. 3672. 3678. 3671. 3660. 3658. 3638. 3641. 3657. 3667.
 3658. 3693. 3675. 3678. 3678. 3703. 3695. 3753. 3676. 3688. 3689. 3748.
 3731. 3766. 3765. 3731. 3712. 3712. 3747. 3753. 3711. 3764. 3729. 3747.
 3763. 3702. 3689. 3710. 3749. 3752. 3752. 3749. 3748. 3747. 3707. 3728.
 3744. 3707. 3721. 3772. 3761. 3752. 3743. 3719. 3747. 3751. 3738. 3760.
 3760. 3715. 3691. 3706. 3698. 3688. 3695. 3697. 3687. 3691. 3711. 3690.
 3696. 3704. 3673. 3672. 3675. 3685. 3686. 3678. 3684. 3682. 3679. 3672.
 3678. 3671. 3666. 3668. 3665. 3671. 3662. 3657. 3667. 3657. 3657. 3660.
 3656. 3656. 3648. 3648. 3653. 3640. 3655. 3653. 3659. 3653. 3643. 3641.
 3641. 3651. 3641. 3642. 3641. 3637. 3638. 3641. 3641. 3640. 3638. 3656.
 3652. 3640. 3641. 3735. 3688. 3657. 3769. 3768. 3757. 3757. 3684. 3754.
 3688. 3662. 3660. 3683. 3676. 3671. 3649. 3659. 3678. 3680. 3681. 3690.
 3671. 3678. 3671. 3667. 3665. 3667. 3665. 3661. 36

In [71]:
A.allocation

{0: [1977, 812, 577, 282],
 1: [1946, 812, 580, 283, 37],
 2: [1928, 814, 585, 286, 47],
 3: [1913, 815, 587, 294, 58],
 4: [1902, 815, 587, 295, 74],
 5: [1886, 816, 589, 300, 103],
 6: [1870, 816, 592, 302, 173],
 7: [1863, 816, 594, 304, 204],
 8: [1858, 818, 596, 310, 170],
 9: [1848, 819, 598, 313, 201],
 10: [1845, 820, 603, 310, 198],
 11: [1823, 820, 609, 336, 124],
 12: [1793, 820, 609, 360, 168],
 13: [1785, 822, 610, 373, 107],
 14: [1755, 823, 613, 399, 106],
 15: [1734, 823, 615, 429, 71],
 16: [1733, 826, 614, 422, 83],
 17: [1722, 829, 619, 431, 70],
 18: [1714, 830, 622, 447, 47],
 19: [1705, 831, 626, 462, 34],
 20: [1677, 834, 631, 496],
 21: [1668, 834, 643, 488, 8],
 22: [1654, 835, 672, 463, 33],
 23: [1650, 835, 682, 441, 59],
 24: [1649, 836, 681, 447, 45],
 25: [1630, 840, 720, 401, 102],
 26: [1629, 844, 712, 412, 78],
 27: [1626, 845, 715, 410, 82],
 28: [1625, 845, 718, 408, 82],
 29: [1620, 847, 725, 397, 114],
 30: [1619, 848, 724, 399, 105],
 31: [1614, 84

In [72]:
A.switchMachine(1977,0,1)

In [66]:
A.workload[0]

3663.0

In [73]:
A.allocation[1]

[1946, 812, 580, 283, 37, 1977]

In [ ]:
# determines whether the solution found by A is indeed feasible, input: A is an 'agent' object
def verifyFeasibleSolution(A):
    # check that each job is assigned to exactly one machine
    assert(len(A.machine) == len(p))
    # check that there are at most m machines that have jobs assigned to them
    assert(max(A.machine) <= m)

    # check that the workloads are as indicated in A.workload
    workload = np.zeros(m)
    for job in range(len(p)):
        workload[A.machine[job]] += p[job]
    for i in range(m):
        assert(workload[i] == A.workload[i])

    # check that the maximum of the workloads (i.e. the cost) is as indicated in A.cost
    assert(np.isclose(A.cost, np.max(A.workload)))

verifyFeasibleSolution(A)